In [1]:
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import json

In [45]:
df = pd.read_csv("List_of_FIFA_World_Cup_finals_2.csv")
df["Winners"] = df["Winners"].replace({"West Germany": "Germany"})
years = df["Year"].unique()
marks = {int(year): str(year) for year in years} 
win_counts = df["Winners"].value_counts().reset_index()
win_counts.columns = ["Country", "Times won"]


df = df.merge(win_counts, left_on="Winners", right_on="Country", how="left")

app = Dash(__name__)
server = app.server

app.layout = html.Div([
    html.H1("World Cup Winners"),
    dcc.Dropdown(df["Winners"].unique(), "Brazil", id='my-dropdown'),
    dcc.Slider(
        min=min(years), 
        max=max(years), 
        step=None, 
        value=1930, 
        marks=marks,
        id='my-slider'),
    dcc.RadioItems(
        id='input-mode',
        options=[
            {'label': 'View all Winners', 'value': 'all'},
            {'label': 'Filter by Country', 'value': 'country'},
            {'label': 'Filter by Year', 'value': 'year'},
        ],
        value='country',
        labelStyle={'display': 'inline-block'}
    ),
    dcc.Graph(id='winners-map')
])

@app.callback(
    Output('winners-map', 'figure'),
    Input('my-dropdown','value'),
    Input('my-slider','value'),
    Input('input-mode', 'value')
)

def update_graph(country, year, filter_mode):
    with open('countriesgeo.json') as data:
        countries = json.load(data)
    
    if filter_mode == 'year':
        filtered = df[df["Year"] == year]
        
        runner_up_fig = px.choropleth(
            filtered,
            geojson = countries,
            locations="Runners-up",
            featureidkey="properties.name",
            hover_name="Runners-up",
            color="Runners-up", 
            color_continuous_scale="Blues", 
            title="World Cup Runner-Ups",
            labels={"Runners-up": "Runner-up"},
        )

        fig = px.choropleth(
            filtered,
            geojson = countries,
            locations="Winners",
            featureidkey="properties.name",
            hover_name="Winners",
            color="Winners", 
            color_continuous_scale="YlOrRd", 
            title="World Cup Winners",
            labels={"Winners": "Winner"}
        )
        fig.add_trace(runner_up_fig.data[0])

    elif filter_mode == "all":
        fig = px.choropleth(
            df,
            geojson = countries,
            locations="Winners",
            featureidkey="properties.name",
            hover_data=["Times won"],
            color="Times won",
            color_continuous_scale="YlOrRd", 
            labels={"Times won": "Number of Wins"}
        )
    elif filter_mode == 'country':
        filtered = df[df["Winners"] == country]   
        fig = px.choropleth(
            filtered,
            geojson = countries,
            locations="Winners",
            featureidkey="properties.name",
            hover_name="Winners",
            color="Winners", 
            color_continuous_scale="earth", 
        )
    else:
        filtered = df
        fig = px.choropleth(
            filtered,
            geojson = countries,
            locations="Winners",
            featureidkey="properties.name",
            hover_name="Winners",
            color="Winners", 
            color_continuous_scale="earth", 
        )
    
    fig.update_geos(fitbounds="locations")

    return fig
    
if __name__ == '__main__':
    app.run(debug=True)